## Eliptic curve for cryptography

### 1. Cơ sở lý thuyết

### 2. Code demo

Đầu tiên, ta định nghĩa class ECC:

In [7]:
class EC:
    a = 'null'
    b = 'null'
    p = 'null'
    def __init__(self, a, b, p):
        if 4*(a**3)+27*(b**2) == 0:
            raise ValueError("4a^3 + 27b^2 must be non zero value")
        self.a = a
        self.b = b
        self.p = p
    def __str__(self):
        return f"ECC: (a, b) = ({self.a}, {self.b}); p = {self.p}"
    def calc(self, x):
        return (x**3 + self.a*x + self.b) % self.p
class ECP:
    def __init__(self, x, y, EC):
        self.x = x
        self.y = y
        self.EC = EC
    def __str__(self):
        return f'({self.x}, {self.y})'
    def __add__(self, Q):
        # Point at infinity
        O = ECP(0, 0, self.EC)
        p = self.EC.p
        a = self.EC.a
        # Get P and Q coordinate
        xp, yp = self.x, self.y
        xq, yq = Q.x, Q.y
        
        if xp == 0 and yp == 0:
            return Q
        if xq == 0 and yq == 0:
            return self
        if xp == xq and (yp + yq) % p == 0:
            return O
        
        if xp == xq and yp == yq:
            s = ((3*(xp**2) + a) * pow(2*yp, -1, p)) % p
        else:
            s = ((yp - yq) * pow((xp - xq), -1, p)) % p
        
        # Calculate R
        xr = (s**2 - xp - xq) % p
        yr = (-yp + s*(xp - xr)) % p
        R = ECP(xr, yr, self.EC)
        return R
    def __mul__(self, n):
        Q = self
        R = ECP(0, 0, self.EC)
        while n > 0:
            if n % 2 == 1:
                R = R + Q
            Q = Q + Q
            n = n // 2
        return R
    def __rmul__(self, n):
        return self * n

Ví dụ minh hoạ:

In [84]:
ec = EC(1, 2, 11)
print(ec)

ECC: (a, b) = (1, 2); p = 11


In [91]:
p = ECP(3, 4, ec)
q = ECP(7, 5, ec)
print('p + 2*q =', p + 2*q)
print('3*q + q*5 + 11*q =', 3*q + q*5 + 11*q)
print('19*q =', 19*q)

p + 2*q = (8, 7)
3*q + q*5 + 11*q = (2, 9)
19*q = (2, 9)


Cho $E = \{Y^2 = X^3 + 497 X + 1768\}, p = 9739, G = (1804,5368)$  
Giả sử Alice có khoá $n_A$, Bob có khoá $n_B$ và 2 người thoả thuận về $E, p, G$ như trên.  
Alice và Bob trao đổi tin như sau: 
* Alice sẽ tính $Q_A = n_A G$, còn Bob sẽ tính $Q_B = n_B G$
* Sau đó Alice gửi $Q_A$ cho Bob, còn Bob gửi $Q_B$ cho Alice  
* Ta gọi $S=n_B Q_A = n_A Q_B$. $S$ là tin nhắn bí mật (*shared secret*) được chia sẻ giữa 2 người  

Giả sử ở phía Bob nhận được thông tin như sau: $Q_A = (815, 3190)$, $n_B = 1829$

In [8]:
# Thiết lập các biến
ec = EC(497, 1768, 9739)
G = ECP(1804, 5368, ec)
Q_a = ECP(815, 3190, ec)
nb = 1829
S = Q_a * nb
print(S)
print(nb*G)

(7929, 707)
(4221, 4874)


In [12]:
Q_x = 4726
nb = 6534
Q_y = pow(ec.calc(Q_x),-2,ec.p)
Q_a = ECP(Q_x, Q_y, ec)
S = Q_a * nb
print(S)

(5796, 9371)
